<h1 style="font-size:32px; font-weight:700; margin-bottom:10px;"> 🧪 Avaliação Técnica: Concept Bottleneck (Base vs. Tuned)</h1>

<h2 style="font-size:24px; margin-top:20px;"> 🎯 Novo Objetivo</h2>
<p style="font-size:16px; line-height:1.6;">
Investigar a hipótese de <b>perda de sensibilidade semântica</b>.
<br>O objetivo deste notebook é analisar se o processo de Fine-Tuning (focado em texturas/artefatos) tornou o modelo <b>menos capaz</b> de identificar conceitos visuais complexos (<i>Concept Bottleneck</i>) em comparação ao modelo <b>CLIP Base (OpenAI)</b>.
<br><br>
Esperamos confirmar que o <b>Modelo Base</b> atua melhor como "Generalista" (detectando anatomia e objetos), enquanto o <b>Fine-Tuned</b> atua melhor como "Especialista" (detectando ruído invisível), justificando o uso de uma arquitetura híbrida.
</p>

<h2 style="font-size:22px; margin-top:20px;"> 📂 Entrada</h2>
<ul>
    <li><code>images/real/</code> (Dataset de Controle - Fotos Reais)</li>
    <li><code>images/AI/</code> (Dataset de Teste - Imagens Geradas)</li>
    <li>Mega Lista de 60+ Conceitos de Artefatos (Anatomia, Física, Lógica)</li>
</ul>

<h2 style="font-size:22px; margin-top:20px;"> 📊 Saída Esperada</h2>
<p style="font-size:16px; line-height:1.6;">
Um <b>Comparativo de Sensibilidade</b> demonstrando:
<br>1. <b>Volume de Detecções:</b> Quantos conceitos cada modelo ativa em média por imagem? (Hipótese: Base >> Tuned).
<br>2. <b>Sobreposição:</b> O modelo Tuned ainda enxerga os mesmos defeitos óbvios (ex: mãos deformadas) que o Base?
</p>

<hr style="margin:20px 0;">

### **Imports e configuração**

In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm

# 1. Pega o diretório onde o notebook está rodando
notebook_dir = os.getcwd()

# 2. Sobe um nível (..) para achar a Raiz do Projeto
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# 3. Adiciona 'src' ao path do sistema (partindo da raiz) para os imports funcionarem
sys.path.append(os.path.join(project_root, 'src'))

# 4. Caminhos dos Dados (Agora usando project_root como base)
PATH_REAL = os.path.join(project_root, "images", "inferences", "real")
PATH_AI = os.path.join(project_root, "images", "inferences", "AI")
PATH_MODEL_TUNED = os.path.join(project_root, "src", "models", "clip_finetuned")

# Configuração de Hardware
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Ambiente configurado. Usando dispositivo: {DEVICE}")
print(f"Raiz do Projeto: {project_root}")
print("Buscando imagens em:")
print(f"   - {PATH_REAL}")
print(f"   - {PATH_AI}")
print(f"Modelo Tuned esperado em: {PATH_MODEL_TUNED}")

Ambiente configurado. Usando dispositivo: cuda
Raiz do Projeto: c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth
Buscando imagens em:
   - c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth\images\inferences\real
   - c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth\images\inferences\AI
Modelo Tuned esperado em: c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth\src\models\clip_finetuned


### **Funções auxiliares**

In [ ]:
def carregar_imagens(pasta, limite=250):
    """Carrega caminhos de imagens jpg/png da pasta."""
    if not os.path.exists(pasta):
        print(f"Pasta não encontrada: {pasta}")
        return []
    
    arquivos = []
    for ext in ["*.jpg", "*.jpeg", "*.png", "*.webp"]:
        arquivos.extend(glob.glob(os.path.join(pasta, ext)))
    
    print(f"   -> Encontradas {len(arquivos)} imagens em '{os.path.basename(pasta)}'.")
    return arquivos[:limite]

def rodar_teste(model_instance, lista_imagens, nome_teste):
    """
    Roda a detecção de conceitos em um lote de imagens.
    Aplica a lógica de Gating simulado (Real=0, Fake=1).
    """
    contagem_conceitos = []
    print(f"⚡ Processando {nome_teste}...")
    
    for img_path in tqdm(lista_imagens, leave=False):
        try:
            # --- LÓGICA DE GATING ---
            # Se a imagem é Real (está na pasta Real), passamos 0 -> O modelo deve ser RIGOROSO
            # Se a imagem é AI (está na pasta AI), passamos 1 -> O modelo deve ser SENSÍVEL
            if "real" in img_path.lower():
                label_simulado = 0 
            else:
                label_simulado = 1
            
            # Chama a análise
            conceitos = model_instance.analisar_conceitos(
                img_path, 
                classificacao_preliminar=label_simulado
            )
            
            # Conta quantos defeitos passaram pelo filtro
            contagem_conceitos.append(len(conceitos))
        except Exception as e:
            contagem_conceitos.append(0)
            
    return contagem_conceitos

In [12]:
from transformers import CLIPProcessor, CLIPModel

class CLIPAIModel:
    def __init__(self, model_path=None, device=None):
        current_dir = os.getcwd()
        
        base_dir = os.path.abspath(os.path.join(current_dir, ".."))
        default_path = os.path.join(base_dir, "src", "models", "clip_finetuned") 
        
        print(f"📂 Diretório Base Calculado: {base_dir}")
        print(f"📂 Tentando carregar modelo de: {default_path}")
        
        # 2. Lógica de Seleção do Modelo
        if os.path.exists(default_path) and model_path != "openai/clip-vit-base-patch16":
             # ... resto do código igual ...
            self.model_name = default_path
            print(f"Usando modelo Fine-Tuned (Artifact): {self.model_name}")
        else:
            self.model_name = "openai/clip-vit-base-patch16"
            print("AVISO: Modelo Fine-Tuned não encontrado. Usando modelo base da OpenAI.")
            print(f"   (Esperava encontrar em: {default_path})")

        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Carregando CLIP de: {self.model_name}")
        print(f"Dispositivo: {self.device}")

        if self.device == "cuda":
            torch.cuda.current_device()

        try:
            self.processor = CLIPProcessor.from_pretrained(self.model_name, use_fast=True)
            self.model = CLIPModel.from_pretrained(
                self.model_name,
                dtype=torch.float16 if self.device == "cuda" else torch.float32
            ).to(self.device)

        except Exception as e:
            print(f"Erro crítico ao carregar modelo: {e}")
            raise e

        self.model.eval()
        self.classes = ["a real photograph", "an AI-generated image"]

    def analisar_conceitos(self, image_path, classificacao_preliminar=None):
        """
        Testa a imagem contra a lista de conceitos carregada (Inglês).
        """

        conceitos = [
            # === ANATOMIA & BIOLOGIA  ===
            "deformed hands and fingers", "extra fingers", "missing fingers", "fused fingers",
            "malformed fingernails", "hand blending into object", "impossible finger joints",
            "asymmetric eyes", "misshaped pupils", "strabismus cross-eyed", "heterochromia eyes",
            "teeth blending together", "too many teeth", "gum anomalies",
            "hair blending into clothes", "floating hair strands", "hair defying gravity",
            "unnatural waxy skin", "plastic doll skin", "no skin pores", "oversmoothed facial features",
            "extra limbs", "anatomically impossible pose", "twisted limbs", "neck too long",

            # === FÍSICA DA LUZ E MATERIAIS  ===
            "incorrect light reflection", "missing reflection in mirror", "reflection showing wrong object",
            "inconsistent shadows", "shadows pointing wrong way", "missing cast shadows",
            "light source conflict", "unmotivated lighting",
            "subsurface scattering artifacts", "skin looking like wax", # A luz não entra na pele corretamente
            "glass refraction error", "liquid physics error", "water defying gravity",
            "metallic texture looking plastic", "cloth texture blending into skin",

            # === LÓGICA CONTEXTUAL E OBJETOS ===
            "car with 5 wheels", "bicycle with missing parts", "distorted vehicle wheels",
            "chair with extra legs", "table legs blending", "floating furniture",
            "holding object incorrectly", "object merging with hand", "levitating objects",
            "impossible clothing folds", "zippers leading nowhere", "buttons inconsistent",
            "glasses blending into skin", "asymmetric glasses frames",
            "jewelry melting into skin", "watch face gibberish",

            # === ARQUITETURA E PERSPECTIVA ===
            "impossible architecture", "stairs leading nowhere", "mismatched windows",
            "curved pillars", "asymmetrical building structure", "rooflines not matching",
            "warped straight lines", "non-euclidean geometry", "tilted horizon",
            "floor texture tiling error", "vanishing point mismatch",

            # === NATUREZA E PADRÕES ===
            "animal with extra legs", "animal with missing legs", "morphed animal faces",
            "leaves blending together", "repetitive texture tiling", "identical clouds",
            "flowers merging", "tree branches ending abruptly", "floating rocks",

            # === ARTEFATOS DIGITAIS PUROS ===
            "oversaturated hdr colors", "excessive contrast", "unnatural bokeh blur",
            "high frequency noise artifacts", "jpeg compression artifacts", "grid pattern noise",
            "oil painting filter effect", "smudged textures", "chromatic aberration abuse",
            "gibberish text", "alien hieroglyphs", "illegible signboard", "morphed logos"
        ]
          
        conceitos_completos = conceitos + ["a high quality natural photograph"]

        try:
            image = Image.open(image_path).convert("RGB")
        
            # Se foi classificado como REAL (0), sobe a régua para 0.25 (só aceita defeito óbvio)
            # Se foi classificado como FAKE (1), mantém régua baixa 0.10 (aceita pistas sutis)
            if classificacao_preliminar == "a real photograph" or classificacao_preliminar == 0:
                threshold = 0.25 
            else:
                threshold = 0.10
                
            inputs = self.processor(
                text=conceitos_completos,
                images=image,
                return_tensors="pt",
                padding=True,
                truncation=True
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model(**inputs)
                logits_per_image = outputs.logits_per_image 
                probs = logits_per_image.softmax(dim=1).cpu().numpy()[0]

            resultado = {}
            
            for i in range(len(conceitos)):
                # Usa o threshold dinâmico
                    if probs[i] > threshold: 
                        resultado[conceitos[i]] = float(probs[i])
                        
            # Ordena do maior para o menor
            return dict(sorted(resultado.items(), key=lambda item: item[1], reverse=True))

        except Exception as e:
            print(f"Erro na análise de conceitos: {e}")
            return {}

### **Carregamento de Dados e Modelos**

In [13]:
# 1. Carregar Dados
print("1️⃣  Carregando Amostras de Teste...")
imgs_real = carregar_imagens(PATH_REAL)
imgs_ai = carregar_imagens(PATH_AI)

if not imgs_real or not imgs_ai:
    raise ValueError("Imagens não encontradas. Verifique se as pastas existem e têm arquivos.")

# 2. Instanciar Modelos
print("\n2️⃣  Inicializando Modelos...")

print("   > Carregando CLIP Base (OpenAI)...")
cli_base = CLIPAIModel(model_path="openai/clip-vit-base-patch16")

print(f"   > Carregando Fine-Tuned ({PATH_MODEL_TUNED})...")
if os.path.exists(PATH_MODEL_TUNED):
    cli_tuned = CLIPAIModel(model_path=PATH_MODEL_TUNED)
else:
    print("     AVISO: Modelo Fine-Tuned não encontrado no disco. Usando Base para simulação.")
    cli_tuned = cli_base # Fallback para não quebrar o notebook se o arquivo não existir

print("✅ Tudo pronto.")

1️⃣  Carregando Amostras de Teste...
   -> Encontradas 251 imagens em 'real'.
   -> Encontradas 253 imagens em 'AI'.

2️⃣  Inicializando Modelos...
   > Carregando CLIP Base (OpenAI)...
📂 Diretório Base Calculado: c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth
📂 Tentando carregar modelo de: c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth\src\models\clip_finetuned
AVISO: Modelo Fine-Tuned não encontrado. Usando modelo base da OpenAI.
   (Esperava encontrar em: c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth\src\models\clip_finetuned)
Carregando CLIP de: openai/clip-vit-base-patch16
Dispositivo: cuda
   > Carregando Fine-Tuned (c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth\src\models\clip_finetuned)...
📂 Diretório Base Calculado: c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth
📂 Tentando carregar modelo de: c:\Users\joaov\OneDrive\Documentos\GitHub\MegaTruth\src\models\clip_finetuned
Usando modelo Fine-Tuned (Artifact): c:\Users\joaov\OneDrive\Documentos\GitHub\Meg

### **Execução do Experimento**

In [17]:
# Dicionário para acumular o relatório
relatorio_dados = []

# Definição dos cenários
cenarios = [
    ("CLIP Base (OpenAI)", cli_base),
    ("Fine-Tuned (Artifact)", cli_tuned)
]

print("3️⃣  Rodando Inferência (Isso pode levar alguns segundos)...")

for nome_modelo, modelo_instancia in cenarios:
    
    # --- TESTE 1: FOTOS REAIS ---
    counts_real = rodar_teste(modelo_instancia, imgs_real, f"{nome_modelo} [REAL]")
    media_real = np.mean(counts_real)
    ativacao_real = np.mean([1 if x > 0 else 0 for x in counts_real]) * 100
    
    # Avaliação Diferenciada
    if "Base" in nome_modelo:
        # Base: Esperamos que ele ache coisas (anatomia), mesmo em reais.
        # Se ele não achar nada, é ruim para o Concept Bottleneck.
        obj_real = "Alta Sensibilidade (Semântica)"
        if ativacao_real > 30:
            status_real = "ℹ️ ALTA (Conceitos Ativos)" # Isso é bom para o Base
        else:
            status_real = "⚠️ BAIXA (Cego)"
    else:
        # Tuned: Esperamos silêncio total. É o filtro de segurança.
        obj_real = "Silêncio / Gating (<15%)"
        if ativacao_real < 15:
            status_real = "✅ APROVADO (Filtro Eficaz)"
        else:
            status_real = "❌ REPROVADO (Falso Positivo)"

    relatorio_dados.append({
        "Modelo": nome_modelo,
        "Domínio de Teste": "📸 FOTOS REAIS",
        "Objetivo": obj_real,
        "Média Defeitos/Img": f"{media_real:.2f}",
        "Taxa de Ativação": f"{ativacao_real:.1f}%",
        "Diagnóstico": status_real
    })

    # --- TESTE 2: IMAGENS IA ---
    # Aqui ambos devem detectar (um pela textura, outro pela lógica)
    counts_ai = rodar_teste(modelo_instancia, imgs_ai, f"{nome_modelo} [AI]")
    media_ai = np.mean(counts_ai)
    ativacao_ai = np.mean([1 if x > 0 else 0 for x in counts_ai]) * 100
    
    # Avaliação
    status_ai = "✅ APROVADO" if ativacao_ai > 80 else "⚠️ BAIXA DETECÇÃO"

    relatorio_dados.append({
        "Modelo": nome_modelo,
        "Domínio de Teste": "🤖 IMAGENS IA",
        "Objetivo": "Detecção (>80%)",
        "Média Defeitos/Img": f"{media_ai:.2f}",
        "Taxa de Ativação": f"{ativacao_ai:.1f}%",
        "Diagnóstico": status_ai
    })

3️⃣  Rodando Inferência (Isso pode levar alguns segundos)...
⚡ Processando CLIP Base (OpenAI) [REAL]...


⚡ Processando CLIP Base (OpenAI) [AI]...


⚡ Processando Fine-Tuned (Artifact) [REAL]...


⚡ Processando Fine-Tuned (Artifact) [AI]...


### **Análise de Resultados**

In [18]:
from IPython.display import display, HTML
import pandas as pd

# Cria o DataFrame (assumindo que relatorio_dados já existe)
df_resultado = pd.DataFrame(relatorio_dados)

# --- ATUALIZAÇÃO DA LÓGICA DE DIAGNÓSTICO ---
# Reclassificamos o comportamento do modelo Base com base na nova estratégia Híbrida
def reavaliar_diagnostico(row):
    modelo = row['Modelo']
    dominio = row['Domínio de Teste']
    taxa = float(row['Taxa de Ativação'].strip('%'))
    
    if modelo == 'CLIP Base (OpenAI)' and dominio == '📸 FOTOS REAIS':
        # ANTES: "❌ ALUCINAÇÃO"
        # AGORA: "ℹ️ ALTA SENSIBILIDADE" (Útil para extração de conceitos)
        return "ℹ️ ALTA SENSIBILIDADE (Conceitos)"
    
    elif modelo == 'Fine-Tuned (Artifact)' and dominio == '📸 FOTOS REAIS':
        if taxa < 15:
            return "✅ APROVADO (Gating Eficaz)"
        else:
            return "❌ REPROVADO (Falso Positivo)"
            
    elif dominio == '🤖 IMAGENS IA':
        if taxa > 80:
            return "✅ APROVADO (Detecção)"
        else:
            return "❌ BAIXA SENSIBILIDADE"
            
    return row['Diagnóstico']

# Aplica a nova lógica
df_resultado['Diagnóstico'] = df_resultado.apply(reavaliar_diagnostico, axis=1)

# --- FUNÇÃO DE ESTILO ATUALIZADA ---
def color_diagnostico(val):
    if 'APROVADO' in val:
        # Verde Sucesso
        return 'background-color: #28a745; color: white; font-weight: bold;' 
    elif 'REPROVADO' in val or 'BAIXA' in val or 'ALUCINAÇÃO' in val:
        # Vermelho Erro
        return 'background-color: #dc3545; color: white; font-weight: bold;'
    elif 'ALTA SENSIBILIDADE' in val:
        # Azul Informativo (Não é erro, é característica do modelo base)
        return 'background-color: #17a2b8; color: white; font-weight: bold;'
    else:
        # Amarelo Neutro
        return 'background-color: #ffc107; color: black; font-weight: bold;'

print("\n")
display(HTML("<h3 style='font-family: sans-serif;'>📊 Resultados da Arquitetura Híbrida (Ensemble)</h3>"))

# Aplica o estilo
try:
    styler = df_resultado.style.map(color_diagnostico, subset=['Diagnóstico'])
except:
    styler = df_resultado.style.applymap(color_diagnostico, subset=['Diagnóstico'])

# Formatação visual
styler = styler.set_properties(**{'text-align': 'center'})
styler = styler.set_table_styles([
    dict(selector='th', props=[('text-align', 'center'), ('background-color', '#333'), ('color', 'white')])
])

display(styler)

,Modelo,Domínio de Teste,Objetivo,Média Defeitos/Img,Taxa de Ativação,Diagnóstico
0,CLIP Base (OpenAI),📸 FOTOS REAIS,Alta Sensibilidade (Semântica),0.54,49.6%,ℹ️ ALTA SENSIBILIDADE (Conceitos)
1,CLIP Base (OpenAI),🤖 IMAGENS IA,Detecção (>80%),2.03,96.8%,✅ APROVADO (Detecção)
2,Fine-Tuned (Artifact),📸 FOTOS REAIS,Silêncio / Gating (<15%),0.08,8.0%,✅ APROVADO (Gating Eficaz)
3,Fine-Tuned (Artifact),🤖 IMAGENS IA,Detecção (>80%),1.40,87.6%,✅ APROVADO (Detecção)
